In [ ]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn import ensemble
from sklearn.impute import SimpleImputer
from sklearn.model_selection import LeaveOneOut
import catboost
import xgboost
import lightgbm
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import pickle
import tensorflow as tf
import keras
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
seed=75

In [ ]:
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc
def plot_roc_curve(y_train, y_pred_train, y_test, y_pred_test, title):
    fig, ax = plt.subplots()  # This creates a new figure and axes

    for data, label, dataset in [(y_train, y_pred_train, 'Train'), (y_test, y_pred_test, 'Test')]:
        if len(data.shape) > 1:  # Check if it's a multi-class problem
            fpr = dict()
            tpr = dict()
            roc_auc = dict()
            for i in range(data.shape[1]):
                fpr[i], tpr[i], _ = roc_curve(data.iloc[:, i], label[:, i])
                roc_auc[i] = auc(fpr[i], tpr[i])
            
            for i in range(data.shape[1]):
                ax.plot(fpr[i], tpr[i], label=f'ROC curve of {dataset} class {i} (area = {roc_auc[i]:.2f})')
        else:
            # Assuming the second column of label array represents the positive class probabilities
            fpr, tpr, _ = roc_curve(data, label[:, 1])
            roc_auc = auc(fpr, tpr)
            ax.plot(fpr, tpr, label=f'{dataset} ROC curve (area = {roc_auc:.2f})')

    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(f'ROC Curve - {title}')
    ax.legend(loc="lower right")
    return fig, ax  # Return the Figure and Axes

In [ ]:
def predict_print_plot(data_input, X_train, X_test, y_train, y_test, algorithm_name, model):
    # Predict labels for all data, train data, and test data
    y_pred_all = model.predict(data_input)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Predict probabilities for all data, train data, and test data
    y_pred_proba_all = model.predict_proba(data_input)
    y_pred_proba_train = model.predict_proba(X_train)
    y_pred_proba_test = model.predict_proba(X_test)
    

    # Plot ROC curve for both train and test data
    fig,ax=plot_roc_curve(y_train, y_pred_proba_train, y_test, y_pred_proba_test, title=algorithm_name)
    return fig,ax

In [ ]:
###########wrapping classification metrics for later calls##########
def compute_classification_metrics(target, prediction):
    accuracy = accuracy_score(target, prediction)
    f1 = f1_score(target, prediction)
    auc = roc_auc_score(target, prediction)
    return accuracy, f1, auc

def gridsearch(model, param, algorithm_name, X_train, y_train, X_test, y_test):
    grid = GridSearchCV(model, param_grid=param, scoring='f1', cv=10, n_jobs=-1, verbose=-1)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    
    prediction_train = best_model.predict(X_train)
    prediction_test = best_model.predict(X_test)
    
    # Combine train and test predictions and true values
    combined_predictions = np.concatenate([prediction_train, prediction_test])
    combined_true_values = np.concatenate([y_train, y_test])
    
    accuracy_train, f1_train, auc_train = compute_classification_metrics(y_train, prediction_train)
    accuracy_test, f1_test, auc_test = compute_classification_metrics(y_test, prediction_test)
    accuracy_all, f1_all, auc_all = compute_classification_metrics(combined_true_values, combined_predictions)

    print(algorithm_name)
    print('Best Classifier:', grid.best_params_)
    print('--- Training Data ---')
    print('Accuracy:', accuracy_train, 'F1:', f1_train, 'AUC:', auc_train)
    print('--- Test Data ---')
    print('Accuracy:', accuracy_test, 'F1:', f1_test, 'AUC:', auc_test)
    print('--- All Data ---')
    print('Accuracy:', accuracy_all, 'F1:', f1_all, 'AUC:', auc_all)
    
    return best_model, (accuracy_test+f1_test+auc_test)/3


In [ ]:
fl = open(r'./database_high_quality_st.pkl', 'rb')
database_full = pickle.load(fl)
##54 AC 55 ST
data_input_full = database_full.iloc[:, 0:55]

# Convert the target variable to binary based on the threshold
threshold = 1
data_output_full = (database_full.iloc[:, 55] < threshold).astype(int)
X_train, X_test, y_train, y_test = train_test_split(data_input_full, data_output_full, test_size=0.1, random_state=seed, stratify=data_output_full)


In [ ]:
model_SVC = SVC(probability=True)
param_svr = {
     'kernel':['linear', 'poly', 'rbf'],
              'max_iter':[100,200,300,400,500,600,700,800,1000,1100,1200,1300,1400,1500],
          'degree':[2,3,4],
         'gamma':['scale','auto'],
              'coef0':[100,200,300,400,500,600,700,800,1000,1100,1200,1300,1400,1500]
# 'coef0': [100], 'degree': [2], 'gamma': ['auto'], 'kernel': ['rbf'], 'max_iter': [200]
       }
SVR_full,SVR_full_score=gridsearch(model_SVC,param_svr,'Support Vector Regressor',X_train,y_train,X_test,y_test)
predict_print_plot(data_input_full, X_train, X_test, y_train, y_test, 'Support Vector Regressor', SVR_full)

In [ ]:
model_KNeighborsClassifier = KNeighborsClassifier()
param_knr = {
    'n_neighbors':range(1,10),'weights':['uniform','distance'],
             'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
         'leaf_size':[2,10,20,30,40,50,100],
         'p':range(1,10)
#     'algorithm': ['auto'], 'leaf_size': [2], 'n_neighbors': [7], 'p': [5], 'weights': ['distance']
    }
KNR_full,KNR_full_score=gridsearch(model_KNeighborsClassifier,param_knr,'K Nearest Neighbor Regressor',X_train,y_train,X_test,y_test)
predict_print_plot(data_input_full, X_train, X_test, y_train, y_test, 'K Nearest Neighbor Regressor', KNR_full)

In [ ]:
model_LGBMClassifier=LGBMClassifier(random_state=1, verbose=-1)
param_lgbm = {
    'boosting_type':['gbdt','rf'],
    'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
    'subsample':[0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
    'n_estimators':[50,100,200,400],
    'max_depth':[5,7,9,11,13,-1],
    'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
    'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
# 'boosting_type': ['gbdt'], 'learning_rate': [0.14], 'max_depth': [5], 'n_estimators': [100], 'reg_alpha': [1e-05], 'reg_lambda': [0.001], 'subsample': [0.4]
}
LGBM_full,LGBM_full_score=gridsearch(model_LGBMClassifier,param_lgbm,'LightGBM',X_train,y_train,X_test,y_test)
predict_print_plot(data_input_full, X_train, X_test, y_train, y_test, 'LightGBM', LGBM_full)

In [ ]:
model_XGBClassifier=XGBClassifier(random_state=1,verbose=-1)
param_xg={
    'booster':['gbtree'],
    'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
    'n_estimators':[100,200,400],
    'max_depth':[3,5,7,9,11,13,-1],
    'subsample':[0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
    'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
    'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
# 'booster': ['gbtree'], 'learning_rate': [0.18], 'max_depth': [9], 'n_estimators': [100], 'reg_alpha': [0.0001], 'reg_lambda': [0.001], 'subsample': [0.75]
}
XG_full,XG_full_score=gridsearch(model_XGBClassifier,param_xg,'XGBoost',X_train,y_train,X_test,y_test)
predict_print_plot(data_input_full, X_train, X_test, y_train, y_test, 'XGBoost', XG_full)

In [ ]:
model_CatBoostClassifier=CatBoostClassifier(random_state=1, verbose=0)
param_cat = {
    'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2],
    'n_estimators':[100,200,400],
    "boosting_type":["Plain"],
    'max_depth':[5,7,9,11],
    'subsample':[0.4,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
    'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
# 'boosting_type': ['Plain'], 'learning_rate': [0.006], 'max_depth': [9], 'n_estimators': [100], 'reg_lambda': [1e-05], 'subsample': [0.65]
}
CAT_full,CAT_full_score=gridsearch(model_CatBoostClassifier,param_cat,'CatBoost',X_train,y_train,X_test,y_test)
predict_print_plot(data_input_full, X_train, X_test, y_train, y_test, 'CatBoost', CAT_full)

In [ ]:
model_GradientBoostingClassifier = ensemble.GradientBoostingClassifier(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
    'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
    'n_estimators':[50,100,200,400],
    'max_depth':[3,5,7,9,11,13,16],
    'criterion':['friedman_mse','mae','mse'],
    'max_features':['auto','sqrt','log2'],
    'loss':['deviance', 'exponential']
# 'criterion': ['friedman_mse'], 'learning_rate': [0.12], 'loss': ['exponential'], 'max_depth': [5], 'max_features': ['auto'], 'n_estimators': [50]}
GB_full,GB_full_score=gridsearch(model_GradientBoostingClassifier,param_GB,'GradientBoost',X_train,y_train,X_test,y_test)
predict_print_plot(data_input_full, X_train, X_test, y_train, y_test, 'GradientBoost', GB_full)

In [ ]:
###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestClassifier = ensemble.RandomForestClassifier(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
    'n_estimators':[50,100,200,400,None],
    'max_depth':[3,5,7,9,11,None],
    'criterion': ['gini', 'entropy'],
    'max_features':['auto','sqrt','log2']
# 'criterion': ['gini'], 'max_depth': [11], 'max_features': ['auto'], 'n_estimators': [50]
}
RF_full,RF_full_score=gridsearch(model_RandomForestClassifier,param_RF,'Random Forest',X_train,y_train,X_test,y_test)
predict_print_plot(data_input_full, X_train, X_test, y_train, y_test, 'Random Forest', RF_full)

In [ ]:
model_DecisionTreeClassifier = DecisionTreeClassifier(random_state=1)
param_dt={
    'max_depth':[5,6,7,8,9,10,11,None],
    'max_features':['auto','sqrt','log2'],
    'criterion': ['gini', 'entropy'],
    'splitter' : [ "best",'random']

# 'criterion': ['gini'], 'max_depth': [5], 'max_features': ['auto'], 'splitter': ['best']
}
DT_full,DT_full_score=gridsearch(model_DecisionTreeClassifier,param_dt,'Decision Tree',X_train,y_train,X_test,y_test)
predict_print_plot(data_input_full, X_train, X_test, y_train, y_test, 'Decision Tree', DT_full)

In [ ]:
model_AdaBoostClassifier = AdaBoostClassifier(random_state=1)
param_ada={
    'n_estimators':[50,100,200,400,800],
    'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1]

# 'learning_rate': [0.2],  'n_estimators': [800]
}
ADA_full,ADA_full_score=gridsearch(model_AdaBoostClassifier,param_ada,'AdaBoost',X_train,y_train,X_test,y_test)
predict_print_plot(data_input_full, X_train, X_test, y_train, y_test, 'AdaBoost', ADA_full)

In [ ]:
def create_ANN_model_1layer(X, learning_rate, regular_term=0.001, neuron_number=50, drop_out_rate=0):
    regularizer = keras.regularizers.l2(regular_term)
    model = Sequential() 
    model.add(Dense(neuron_number, input_dim=X.shape[1], kernel_initializer='random_normal',
                    bias_initializer='random_normal', activation='relu', kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    model.add(Dense(neuron_number, input_dim=neuron_number, kernel_initializer='random_normal',
                    bias_initializer='random_normal', activation='relu', kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    model.add(Dense(1, activation='sigmoid'))  # Changed activation to sigmoid for binary classification
    adam = optimizers.Adam(learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])  # Changed loss to binary_crossentropy
    return model

def create_ANN_model_2layer(X, learning_rate, regular_term=0.001, neuron_number=50, drop_out_rate=0):
    regularizer = keras.regularizers.l2(regular_term)
    model = Sequential() 
    model.add(Dense(neuron_number, input_dim=X.shape[1], kernel_initializer='random_normal',
                    bias_initializer='random_normal', activation='relu', kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    model.add(Dense(neuron_number, input_dim=neuron_number, kernel_initializer='random_normal',
                    bias_initializer='random_normal', activation='relu', kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    model.add(Dense(neuron_number, input_dim=neuron_number, kernel_initializer='random_normal',
                    bias_initializer='random_normal', activation='relu', kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    model.add(Dense(1, activation='sigmoid'))  # Changed activation to sigmoid for binary classification
    adam = optimizers.Adam(learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])  # Changed loss to binary_crossentropy
    return model

In [ ]:
model_ANNClassifier1= KerasClassifier(build_fn=create_ANN_model_1layer(X=data_input_full,learning_rate=0.01), verbose=0)
model_ANNClassifier2= KerasClassifier(build_fn=create_ANN_model_2layer(X=data_input_full,learning_rate=0.01), verbose=0)

In [ ]:
epochs_list=[]
for i in range(10,210,10):
    epochs_list.append(i)

In [ ]:
# 设置参数候选值
batch_size_list = [8,16,32]
optimizers_list=['sgd', 'rmsprop', 'adam', 'adagrad']
param_ann = dict(batch_size=batch_size_list, 
                 epochs=epochs_list,
                optimizer=optimizers_list
                )

In [ ]:
ANN_1layer,ANN_1layer_score=gridsearch(model_ANNClassifier1,param_ann,'Artificial Neural Network 1 layer',X_train,y_train,X_test,y_test)
predict_print_plot(data_input_full, X_train, X_test, y_train, y_test, 'Artificial Neural Network 1 layer', ANN_1layer)

In [ ]:
ANN_2layer,ANN_2layer_score=gridsearch(model_ANNClassifier2,param_ann,'Artificial Neural Network 2 layer',X_train,y_train,X_test,y_test)
predict_print_plot(data_input_full, X_train, X_test, y_train, y_test, 'Artificial Neural Network 2 layer', ANN_2layer)

In [ ]:
# param_ann_1={'batch_size': [32], 'epochs': [190], 'optimizer': ['adam']}
# param_ann_2={'batch_size': [16], 'epochs': [160], 'optimizer': ['rmsprop']}